In [74]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import math
from unidecode import unidecode

In [79]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html

url='https://www.drogariaveracruz.com.br/medicamentos/'


def num_pag(html):
    string=html.find_all('div',{'class':'text-center pt-3'})[1].text
    lista=string.split()
    paginas= int(lista[-1])
    return paginas

def get_name(html):
    NOMES=[]
    nome=html.find_all('h2',{'class':'title'})

    for elem in nome:
        elem=elem.text
        elem=elem.replace('\n                            ',"")
        elem=elem.replace('\n                        ',"")
        NOMES.append(elem)

    return NOMES


def get_sale_price(html):
    SALE_PRICE=[]
    sale_price=html.find_all('p',{'class':'sale-price'})

    for preço in sale_price:
        preço=preço.text
        preço=preço.replace('R$ ',"")
        preço=float(preço.replace(",","."))
        SALE_PRICE.append(preço)
    return SALE_PRICE


def get_full_price(html):
    FULL_PRICE=[]
    div=html.find_all('div',{'class':'prices'})
    for caixa in div:
        full_price=caixa.find('p',{'class':'unit-price'})
        if full_price==None:
            full_price=caixa.find('p',{'class':'sale-price'})
        full_price=full_price.text
        
        full_price=full_price.replace('\n                                            ',"")
        full_price=full_price.replace('\n                                        ',"")
        full_price=full_price.replace('R$ ','')
        full_price=full_price.replace('.',"")
        full_price=float(full_price.replace(',','.'))


        FULL_PRICE.append(full_price)
    return FULL_PRICE




def get_ean(produto):
    produto=produto.replace(" ","-")
    produto=produto.lower()
    produto=unidecode(produto)
    url=f'https://www.drogariaveracruz.com.br/{produto}/p'

    html=pegar_html(url)

    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})
    if div == None:
        gtin="PROBLEMA COM LINK"
    else:
        tag_gtin=div.find('meta',{'itemprop':'gtin13'})
        gtin=tag_gtin.get('content')
        gtin=int(gtin)
    return gtin




#para pegar o nome da marca, fazer ele entrar na página do produto e procurar em const dataItem={}
def get_marca(produto):
    produto=produto.replace(" ","-")
    produto=produto.lower()
    produto=unidecode(produto)
    url=f'https://www.drogariaveracruz.com.br/{produto}/p'

    html=pegar_html(url)

    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})
    if div == None:
        marca="PROBLEMA COM LINK"
    else:
        tag_marca=div.find('meta',{'itemprop':'brand'})
        marca=tag_marca.get('content')
    return marca


nome='Lactosil 10.000 FCC 30 Tabletes'
html=pegar_html(url)
print(get_ean(nome), get_marca(nome))

PROBLEMA COM LINK PROBLEMA COM LINK
